# Seminar 1

# Extended Euclidean algorithm

We want to find   $b=a^{-1}modf$, in other words $ab=1 modf$ that is equivalent to $fx+ab=1$.

This equation can be solved applying extended Euclidean algorithm for pair (f,a). This can be presented in iterative form below:

Step 0: $r_{-2}=f$, $r_{-1}=a$, $y_{-2}=0$, $y_{-1}=1$ 

Step 1: $r_{-2}=r_{-1}q_{0}+r_{0}$, $y_{0}=y_{-2}-y_{-1}q_{0}$


.....

Repear untill $r_{x}=1$ then $y_{x}=b$




## Some code

In [3]:
#####################################################################
# Note on what these operators do:
# %  is the modulus (remainder) operator: 10 % 3 is 1
# // is integer (round-down) division: 10 // 3 is 3
# ** is exponent (2**3 is 2 to the 3rd power)

def eea(a,b):
    if b==0:return (1,0)
    print("a ",a, "b ",b)
    (q,r) = (a//b,a%b)
    print("q ",q,"r ",r)
    (s,t) = eea(b,r)
    return (t, s-(q*t) )
            
def find_inverse(a,f):
    b = eea(a,f)[0]
    if b < 1: b += f #we only want positive values
    return b            

In [4]:
E=23
T=121

D = find_inverse(E,T)

a  23 b  121
q  0 r  23
a  121 b  23
q  5 r  6
a  23 b  6
q  3 r  5
a  6 b  5
q  1 r  1
a  5 b  1
q  5 r  0


In [5]:
print(D)
print((D*E)%T)

100
1


# Euler’s Totient Function

Euler's totient function counts the positive integers up to a given integer n that are relatively prime to n

Can be computed by busting (see code below)



In [4]:
def gcd(a, b): 
    if (a == 0): 
        return b 
    return gcd(b % a, a) 


def phi(n): 
  
    result = 1
    for i in range(2, n): 
        if (gcd(i, n) == 1): 
            result+=1
    return result 

In [5]:
print(phi(36))

12


Another way for computing Euler's totient function is using property that each number n can be presented as $p_{1}^{k_1}p_{2}^{k_2}...p_{l}^{k_l}$, where $p_{1},...p_{l}$ are prime numbers. 

Then $\phi(n)$=$\phi(p_1)\phi(p_2)...\phi(p_l)=n(1-\frac{1}{p_1})...(1-\frac{1}{p_l})$

 Let us check this $36=2^23^2$ Therefore $\phi(36)=36(1-\frac{1}{2})(1-\frac{1}{3})=12

# Fast power computing algorithm

The method is based on observation that $x^n=x(x^2)^{\frac{n-1}{2}}$ if n is odd and $(x^2)^{\frac{n}{2}}$ and multiplicativity of mod operation. So we can present power as a binary number and make a recursive procedure. In each step corresponding to 1 in binary representation multiple the result of the previous step by x and find the binary power of result and in each step corresponding to 0 find the binary power of result of the previous step. To work with "small" numbers after each step takes a module operation. In code, it's presented below

In [6]:
def exp_by_squaring(x, n, f):
    if (n ==0):
        return  1
    elif (n ==1):
        return  x%f 
    elif (n%2==0): 
        return (exp_by_squaring(x*x,  n/2, f))%f
    elif (n%2==1):
        return (x*exp_by_squaring(x*x, (n - 1)/2, f))%f

In [7]:
print(exp_by_squaring(124, 28,37))

33


## Hashing

See SHA-1 pseudocode below. Graphic representation is available at https://upload.wikimedia.org/wikipedia/commons/thumb/e/e2/SHA-1.svg/365px-SHA-1.svg.png

In [1]:
def sha1(data):
    bytes = ""
    
    #Замечание: Все используемые переменные 32 бита.

    #Инициализация переменных:
    h0 = 0x67452301
    h1 = 0xEFCDAB89
    h2 = 0x98BADCFE
    h3 = 0x10325476
    h4 = 0xC3D2E1F0

    #Сначала добавляется 1 (бит),
    #а потом нули, чтобы длина блока стала равной (512 — 64 = 448) бит.
    for n in range(len(data)):
        bytes+='{0:08b}'.format(ord(data[n]))
    bits = bytes+"1"
    pBits = bits
    #pad until length equals 448 mod 512
    while len(pBits)%512 != 448:
        pBits+="0"
    #append the original length
    pBits+='{0:064b}'.format(len(bits)-1)

    def chunks(l, n):
        return [l[i:i+n] for i in range(0, len(l), n)]

    def rol(n, b):
        return ((n << b) | (n >> (32 - b))) & 0xffffffff

    #В процессе сообщение разбивается последовательно по 512 бит:
    #for перебираем все такие части
    for c in chunks(pBits, 512): 
        words = chunks(c, 32)
        w = [0]*80
        #разбиваем этот кусок на 16 частей, слов по 32-бита (big-endian) w[i], 0 <= i <= 15
        for n in range(0, 16):
            w[n] = int(words[n], 2)
        # 16 слов по 32-бита дополняются до 80 32-битовых слов:
        for i in range(16, 80):
            w[i] = rol((w[i-3] ^ w[i-8] ^ w[i-14] ^ w[i-16]), 1)  
        
        #Инициализация хеш-значений этой части:
        a = h0
        b = h1
        c = h2
        d = h3
        e = h4

        #Main loop
        for i in range(0, 80):
            if 0 <= i <= 19:
                f = (b & c) | ((~b) & d)
                k = 0x5A827999
            elif 20 <= i <= 39:
                f = b ^ c ^ d
                k = 0x6ED9EBA1
            elif 40 <= i <= 59:
                f = (b & c) | (b & d) | (c & d) 
                k = 0x8F1BBCDC
            elif 60 <= i <= 79:
                f = b ^ c ^ d
                k = 0xCA62C1D6

            temp = rol(a, 5) + f + e + k + w[i] & 0xffffffff
            e = d
            d = c
            c = rol(b, 30)
            b = a
            a = temp

        # Добавляем хеш-значение этой части к результату:
        h0 = h0 + a & 0xffffffff
        h1 = h1 + b & 0xffffffff
        h2 = h2 + c & 0xffffffff
        h3 = h3 + d & 0xffffffff
        h4 = h4 + e & 0xffffffff

    # Итоговое хеш-значение(h0, h1, h2, h3, h4 должны быть преобразованы к big-endian):
    return '%08x%08x%08x%08x%08x' % (h0, h1, h2, h3, h4)

Function call example

In [4]:
hex_string = sha1("sha")
print(hex_string)

d8f4590320e1343a915b6394170650a8f35d6926


Let's define several strings.

In [28]:
string_small1 = 'This is a very small string with a few characters.'
string_larger = 'This is a larger string that contains more characters.'
string_big = 'This is a larger string that contains more characters. This demonstrates that no matter how big the input stream is, the generated hash is the same size (but of course, not the same value). If two files have a different hash, they surely contain different data.'
string_empty = ''

And print bit representation of the given hash.

In [11]:
import binascii

binary_string = bin(int(hex_string, 16))
print(binary_string)

0b10101010101110011011000011010111001001010011111100111110110100000101011101101111101001010111110100000010001011100111001110100100011110111010000100011011101101


### Task 1. Calculate hashes of the texts abowe.

In [15]:
string_small=sha1("This is a very small string with a few characters.")
string_larger=sha1("This is a larger string that contains more characters.")
string_big=sha1("This is a larger string that contains more characters. This demonstrates that no matter how big the input stream is, the generated hash is the same size (but of course, not the same value). If two files have a different hash, they surely contain different data.")
string_empty=sha1("")
print(string_small)
print(string_larger)
print(string_big)
print(string_empty)

9873c6011814ced6152de6c83d2629e77d60c993
9ed56fc8a27486308ebbf55b06ceae9cd6b1a3fe
0d9ebf408c72b966dc178765203c649d2d664cf1
da39a3ee5e6b4b0d3255bfef95601890afd80709


### Task 2. What is a bit length of each hash?

In [19]:
print(len(string_small))
print(len(string_larger))
print(len(string_big))

40
40
40


In [24]:
import binascii
string_small_lenght = bin(int(string_small, 16))
string_larger_lenght=bin(int(string_larger, 16))
string_big_lenght=bin(int(string_big, 16))
string_empty_lenght=bin(int(string_empty, 16))
print(len(string_small_lenght))
print(len(string_larger_lenght))
print(len(string_big_lenght))
print(len(string_empty_lenght))

162
162
158
162


Let's change the first character of the small string:

In [27]:
small_string_changed1 = 'this is a very small file with a few characters.'
small_string_changed=sha1("this is a very small file with a few characters.")

### Task 3. What is the bitwise distance between two small files? What is bitwise distance between their hashes?

In [37]:
zipped=zip(small_string_changed1,string_small1)
#list(zipped)
d=0
for i in zipped:
    if i[0]!=i[1]:
        d+=1
print(d)

26


In [48]:
zipp_hash=zip(small_string_changed,string_small)

d=0
for i in zipp_hash:
    if i[0]!=i[1]:
        d+=1
print(d)



40
